In [2]:
import pandas as pd

# Задание 1 #

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [184]:
rt = pd.read_csv('ml-latest-small/ratings.csv')
print(rt.head())

res = pd.pivot_table(rt, index='userId', values=['movieId', 'timestamp'], aggfunc={'movieId': 'size', 'timestamp': [min, max]})
res=res[res.movieId['size']>100]
res['ltv_sec'] = res.apply(lambda x: x.timestamp['max']-x.timestamp['min'], axis=1)

# время же в секундах, так?
print(f"\nСреднее время жизни в днях: {round(res.mean()['ltv_sec']/60/60/24,2)}")

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205

Среднее время жизни в днях:     463.89
dtype: float64


# Задание 2#

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [185]:
# rzd - железнодорожные перевозки
# auto - автомобильные перевозки
# air - воздушные перевозки
# client_base - адреса клиентов

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
       'client_id': [115, 116, 117, 118],
       'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [186]:
# возможно я что-то не так понял в задании, но исходя из таблиц данные нормализированы, агрегированы
# и доп. группировок не требуют
mrg = (client_base
.merge(rzd, on='client_id', how='left').fillna(0)
.merge(auto, on='client_id', how='left').fillna(0)
.merge(air, on='client_id', how='left').fillna(0)
)
mrg_sm = mrg.drop(columns='address')

print(mrg_sm, '\n')
mrg_sm

   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           0.0          0.0
1        112       2810.0           0.0          0.0
2        113      10283.0       57483.0          0.0
3        114       5774.0          83.0          0.0
4        115        981.0         912.0         81.0
5        116          0.0        4834.0          4.0
6        117          0.0          98.0         13.0
7        118          0.0           0.0        173.0 



,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


# Задание 3# 

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей.Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить? 
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [ ]:
# Конечно очень умозрительные вопросы, ответы на которые зависят от имеющейся структуры данных
# И мне не совсем понятно, что мы хотим увидеть от этих координат? Зачем они  нужны (нам как аналитикам или заказчику)?
# Ну, исходим из того, что мы хотим знать, где физически был клиент, когда что-то заказывал.

* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Предполагаю, что в обеих таблицах должен быть некий time_stamp события. Привязка делается по ID, из базы координат выбирается нужное время (если их несколько, то фильтруется по первому или последнему значению). Т.о. у нас будет одни координаты/локация для покупок или визитов.

* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Интересный вопрос. Отразится это снижением достоверности данных, как и всегда. Поскольку среднее (координат) нам в данном случае мало что скажет, равно как и все зафиксированные пункты по пути, то отстается только выбрать одну из записей. Обычно это первая или последняя, попадающая в нужный временной период. Опять же, обычно (в моей практике) берется последняя запись.

* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Из них можно и нужно вытащить географическую локацию через соотв. справочник.
Понять что координаты входят в справочник - непростая задача. Зависит от заложенной топографии и точности. В простом случае можно считать квадратиками (в справочнике координаты локации от до по долготе и широте). Вопрос, где взять этот справочник или кто его будет составлять :)
Скорее всего, есть соотв. он-лайн сервисы, к которым можно присосаться (но это будет дооооолго для большой базы).

Итого:
* Нам нужен справочник географических локаций (населенных пунктов). Поскольку упоминается таблица визитов (физических? или на сайт?), то этот справочник можно ограничить локациями (городами), где у нас есть магазины. Если нужна большая точностьо можно детализировать до районов города, где есть магазин. Всё непопавшее в диапазоны можно считать как "unknown location"
* Связываем таблицу координат со справочником. Укрупняем timestamp до 1 часа/дня для лучшей привязки к таблицам визитов и покупок (зависит от структуры и точности данных).
* Связываем таблицу визитов и покупок с таблицей координат, куда мы уже подтянули справочник локаций - по id и укрeпненному времени. При наличии нескольких записей отбираем одну (например, первую, если соритируем по времени события desc + найденным локациям в справочнике).
Далее, у нас появляется привязка абонента к локациям, в которых он делал визиты или покупки. Можно посмотреть, где нас больше всего любят/тратят денег/заходят но не платят и т.п.